<a href="https://colab.research.google.com/github/TianyiZhang-zzz/SupplyChain/blob/main/Alko_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Alko case, Supply Chain Analytics, ISOM 599R Emory University

In [1]:
%reset -f
%clear
import numpy as np
import pandas as pd
import math
import scipy.stats as st
from scipy.optimize import minimize

### Set display options for comma separator and precision
pd.options.display.float_format = '{:,.2f}'.format

# Define the date you want to check
days = 365  # days_per_year
h   = 0.15  # holding_cost_per_day
tc  = 0.19  # transportaion_cost_per_unit
ctc = 0.29  # centralized transportaion_cost_per_unit
L   = 5     # supply_leadtime_days
T   = 6     # review_period_days
csl = 0.95  # critical value
# warehouse_cost = 50000

corr = 0
# Prompt the user to enter a value for 'corr'
#corr = float(input("Enter correlation (as scalar): "))

### Rawdata: average demand and standard deviation of demand for each part in each region
avg_demand = np.array([[35.48, 22.61, 17.66, 11.81,  3.36],
                       [ 2.48,  4.15,  6.15,  6.16,  7.49],
                       [ 0.48,  0.73,  0.80,  1.94,  2.54]])

std_demand = np.array([[ 6.98,  6.48,  5.26,  3.48,  4.49],
                       [ 3.16,  6.20,  6.39,  6.76,  3.56],
                       [ 1.98,  1.42,  2.39,  3.76,  3.98]])

num_demand = np.array([[ 10, 10, 10, 10, 10],
                       [ 20, 20, 20, 20, 20],
                       [ 70, 70, 70, 70, 70]])

n_parts, n_regions = avg_demand.shape[0], avg_demand.shape[1]

### create correlation matrix using provided scalar value (corr can also be provided as raw data)
corr_matrix = np.full((n_regions, n_regions), corr)
np.fill_diagonal(corr_matrix, 1) # Set diagonal elements to 1

%clear

# Objective function to maximize (using matrix calculations)
def objective_function(flat_matrix):

    centralize = flat_matrix.reshape((3, 5))     # Reshape the flattened matrix to 3x5

    ### Compute aggregate standard deviation for centralized choices
    agg_std_demand = np.diag( np.dot( np.dot( np.multiply(std_demand, centralize), corr_matrix), np.transpose( np.multiply(std_demand, centralize) ) ) )
    sqrt_agg_std_demand = np.sqrt(agg_std_demand)

    sum_columns = np.sum(centralize, axis=1, keepdims=True)
    sum_columns[sum_columns == 0] = 1
    sum_columns_inv = 1 / sum_columns

    ### Adjust the standard deviation by dividing number of centralized regions for each part
    sqrt_agg_std_demand2 = np.repeat(sqrt_agg_std_demand.reshape(n_parts, 1), n_regions, axis=1)
    sum_columns_inv2 = np.repeat(sum_columns_inv.reshape(n_parts, 1), n_regions, axis=1)

    std_demand_updated = np.array( centralize * sum_columns_inv2 * sqrt_agg_std_demand2 + (1 - centralize) * std_demand )

    total_cost = sum(
        days * num_demand[i, j] *
        (h * ( st.norm.ppf(csl) * std_demand_updated[i, j] * math.sqrt(T + L) +  avg_demand[i, j] * (T/2 + L) )
        + ( avg_demand[i, j] * ( centralize[i, j] * ctc + (1 - centralize[i, j]) * tc ) )
        ) for i in range(n_parts) for j in range(n_regions) )

    return total_cost  # Negate for maximization




Correlation : 0
Minimum cost: $ 1,272,712.13
CDC cost    : $   561,114.50
Investment  : $ 1,041,671.75

Optimal Solution:
   R1   R2   R3   R4   R5
   0    1    1    1    1
   1    1    1    1    1
   1    1    1    1    1


In [1]:
import numpy as np
import scipy.stats as st

# Parameters
days = 365                    # Days per year
h = 0.15                      # Holding cost per unit per day
tc = 0.19                     # Transportation cost per unit
L = 5                         # Lead time (days)
T = 6                         # Review period (days)
z = st.norm.ppf(0.95)         # z value for 95% CSL

# Demand Data (daily)
avg_demand = np.array([
    [35.48, 22.61, 17.66, 11.81,  3.36],
    [ 2.48,  4.15,  6.15,  6.16,  7.49],
    [ 0.48,  0.73,  0.80,  1.94,  2.54]
])

std_demand = np.array([
    [6.98, 6.48, 5.26, 3.48, 4.49],
    [3.16, 6.20, 6.39, 6.76, 3.56],
    [1.98, 1.42, 2.39, 3.76, 3.98]
])

# Number of stores per region and part
num_demand = np.array([
    [10, 10, 10, 10, 10],
    [20, 20, 20, 20, 20],
    [70, 70, 70, 70, 70]
])

n_parts, n_regions = avg_demand.shape

# Initialize totals
total_inventory_cost = 0
total_transport_cost = 0

# Loop through parts and regions
for i in range(n_parts):
    for j in range(n_regions):
        μ = avg_demand[i, j]
        σ = std_demand[i, j]
        n = num_demand[i, j]

        # Periodic review formulas
        ss = z * σ * np.sqrt(T + L)                     # Safety Stock
        cycle = μ * (T / 2 + L)                         # Cycle Stock
        inv_cost = h * (ss + cycle) * days * n          # Annual Inventory Cost
        trans_cost = μ * tc * days * n                  # Annual Transport Cost

        total_inventory_cost += inv_cost
        total_transport_cost += trans_cost

# Add fixed warehouse cost ($50,000 per region × 5 regions)
warehouse_cost = 5 * 50000

# Final total cost
total_cost = total_inventory_cost + total_transport_cost + warehouse_cost

# Print results
print(f"Annual Inventory Cost     : ${total_inventory_cost:,.2f}")
print(f"Annual Transportation Cost: ${total_transport_cost:,.2f}")
print(f"Warehouse Cost            : ${warehouse_cost:,.2f}")
print(f"\n➡️ Total Annual Cost       : ${total_cost:,.2f}")


Annual Inventory Cost     : $1,347,070.96
Annual Transportation Cost: $131,217.14
Warehouse Cost            : $250,000.00

➡️ Total Annual Cost       : $1,728,288.09


In [5]:
import numpy as np
import scipy.stats as st

# Parameters
days = 365
h = 0.15                  # Holding cost/day
ctc = 0.29                # Central transportation cost/unit
L = 5
T = 6
z = st.norm.ppf(0.95)     # CSL = 0.95

# Demand data
avg_demand = np.array([
    [35.48, 22.61, 17.66, 11.81, 3.36],
    [2.48, 4.15, 6.15, 6.16, 7.49],
    [0.48, 0.73, 0.80, 1.94, 2.54]
])

std_demand = np.array([
    [6.98, 6.48, 5.26, 3.48, 4.49],
    [3.16, 6.20, 6.39, 6.76, 3.56],
    [1.98, 1.42, 2.39, 3.76, 3.98]
])

num_demand = np.array([
    [10, 10, 10, 10, 10],
    [20, 20, 20, 20, 20],
    [70, 70, 70, 70, 70]
])

# Function to compute centralized cost under correlation rho
def centralized_cost(rho):
    total_inventory_cost = 0
    total_transport_cost = 0
    total_volume = 0
    n_parts, n_regions = avg_demand.shape

    for i in range(n_parts):
        μ_total = np.sum(avg_demand[i])
        σs = std_demand[i]
        μs = avg_demand[i]

        # Compute pooled standard deviation
        pooled_variance = np.sum(σs**2) + rho * np.sum([
            σs[m] * σs[n] for m in range(n_regions) for n in range(n_regions) if m != n
        ])
        σ_pooled = np.sqrt(pooled_variance)

        # Inventory and transport cost (apply for all units of that part)
        n_units = np.sum(num_demand[i])
        cycle = μ_total * (T / 2 + L)
        ss = z * σ_pooled * np.sqrt(T + L)

        inv_cost = h * (cycle + ss) * days * n_units
        trans_cost = μ_total * ctc * days * n_units

        total_inventory_cost += inv_cost
        total_transport_cost += trans_cost
        total_volume += μ_total * n_units * days

    # CDC investment
    if total_volume <= 400000:
        investment = 2 * total_volume
    else:
        investment = 800000 + 1.5 * (total_volume - 400000)

    total_cost = total_inventory_cost + total_transport_cost + investment

    print(f"\n========== Correlation (ρ) = {rho} ==========")
    print(f"Total Inventory Cost : ${total_inventory_cost:,.2f}")
    print(f"Total Transport Cost : ${total_transport_cost:,.2f}")
    print(f"CDC volume           : {total_volume:,.0f}")
    print(f"Investment           : ${investment:,.2f}")
    print(f"➡️ Total Centralized Cost : ${total_cost:,.2f}")

# Run for ρ = 0, 0.5, 1
for rho in [0, 0.5, 1]:
    centralized_cost(rho)



========== Correlation (ρ) = 0 ==========
Total Inventory Cost : $5,364,373.90
Total Transport Cost : $1,001,393.92
CDC volume           : 3,453,082
Investment           : $5,379,623.75
➡️ Total Centralized Cost : $11,745,391.58

========== Correlation (ρ) = 0.5 ==========
Total Inventory Cost : $6,169,405.69
Total Transport Cost : $1,001,393.92
CDC volume           : 3,453,082
Investment           : $5,379,623.75
➡️ Total Centralized Cost : $12,550,423.37

========== Correlation (ρ) = 1 ==========
Total Inventory Cost : $6,735,354.79
Total Transport Cost : $1,001,393.92
CDC volume           : 3,453,082
Investment           : $5,379,623.75
➡️ Total Centralized Cost : $13,116,372.47


In [6]:
import numpy as np
import math
from itertools import product

# -----------------------
# 1. INPUT DATA & SETUP
# -----------------------

categories = {
    'High': {
        'n_skus': 10,
        'avg_demand': np.array([35.48, 22.61, 17.66, 11.81, 3.36]),
        'std_demand': np.array([6.98, 6.48, 5.26, 3.48, 4.49])
    },
    'Medium': {
        'n_skus': 20,
        'avg_demand': np.array([2.48, 4.15, 6.15, 6.16, 7.49]),
        'std_demand': np.array([3.16, 6.20, 6.39, 6.76, 3.56])
    },
    'Low': {
        'n_skus': 70,
        'avg_demand': np.array([0.48, 0.73, 0.80, 1.94, 2.54]),
        'std_demand': np.array([1.98, 1.42, 2.39, 3.76, 3.98])
    }
}

# Periodic Review Parameters
T = 6
L = 5
TL = T + L
z = 1.645
h = 0.15
days_per_year = 365

# Transportation Costs
dc_ship = 0.09 + 0.10   # 0.19
ndc_ship = 0.05 + 0.24  # 0.29

# ------------------------
# 2. COST FUNCTIONS
# ------------------------

def annual_cost_decentralized(categories_dict):
    total = 0
    for cat in categories_dict.values():
        n = cat['n_skus']
        mu, sigma = cat['avg_demand'], cat['std_demand']
        for j in range(5):
            cycle = mu[j] * (T/2 + L)
            safety = z * sigma[j] * math.sqrt(TL)
            hold = h * (cycle + safety) * days_per_year
            ship = mu[j] * dc_ship * days_per_year
            total += (hold + ship) * n
    return total

def annual_cost_centralized(categories_dict, corr=0.0):
    corr_mat = np.full((5, 5), corr)
    np.fill_diagonal(corr_mat, 1.0)
    total = 0
    for cat in categories_dict.values():
        n = cat['n_skus']
        mu, sigma = cat['avg_demand'], cat['std_demand']
        mu_total = np.sum(mu)

        cov = np.zeros((5,5))
        for i in range(5):
            for j in range(5):
                cov[i,j] = sigma[i]*sigma[j]*corr_mat[i,j] if i != j else sigma[i]**2
        pooled_std = np.sqrt(np.ones(5).dot(cov).dot(np.ones(5)))

        cycle = mu_total * (T/2 + L)
        safety = z * pooled_std * math.sqrt(TL)
        hold = h * (cycle + safety) * days_per_year
        ship = mu_total * ndc_ship * days_per_year
        total += (hold + ship) * n
    return total

def annual_cost_hybrid(categories_dict, centralize_set, corr=0.0):
    corr_mat = np.full((5,5), corr)
    np.fill_diagonal(corr_mat, 1.0)
    total = 0
    for name, cat in categories_dict.items():
        n = cat['n_skus']
        mu, sigma = cat['avg_demand'], cat['std_demand']
        if name in centralize_set:
            mu_total = np.sum(mu)
            cov = np.zeros((5,5))
            for i in range(5):
                for j in range(5):
                    cov[i,j] = sigma[i]*sigma[j]*corr_mat[i,j] if i != j else sigma[i]**2
            pooled_std = np.sqrt(np.ones(5).dot(cov).dot(np.ones(5)))

            cycle = mu_total * (T/2 + L)
            safety = z * pooled_std * math.sqrt(TL)
            hold = h * (cycle + safety) * days_per_year
            ship = mu_total * ndc_ship * days_per_year
            total += (hold + ship) * n
        else:
            for j in range(5):
                cycle = mu[j] * (T/2 + L)
                safety = z * sigma[j] * math.sqrt(TL)
                hold = h * (cycle + safety) * days_per_year
                ship = mu[j] * dc_ship * days_per_year
                total += (hold + ship) * n
    return total

# ------------------------
# 3. Q1: FULLY DECENTRALIZED
# ------------------------

cost_q1 = annual_cost_decentralized(categories)
print(f"\n Q1 - Fully Decentralized Cost: ${cost_q1:,.2f}")

# ------------------------
# 4. Q2: FULL CENTRALIZATION under different rho
# ------------------------

for rho in [0, 0.5, 1.0]:
    cost_q2 = annual_cost_centralized(categories, corr=rho)
    print(f"\n Q2 - Full Centralization Cost (ρ = {rho}): ${cost_q2:,.2f}")
    print(f"   Savings vs Decentralized: ${cost_q1 - cost_q2:,.2f}")

# ------------------------
# 5. Q3: PARTIAL CENTRALIZATION (Best of 8 options)
# ------------------------

all_categories = list(categories.keys())
cost_best = float('inf')
best_set = set()

for combo in product([True, False], repeat=3):
    selected = {cat for cat, flag in zip(all_categories, combo) if flag}
    cost = annual_cost_hybrid(categories, selected, corr=0.0)
    if cost < cost_best:
        cost_best = cost
        best_set = selected

print(f"\n Q3 - Hybrid Strategy (ρ = 0):")
print(f"   Best categories to centralize: {best_set if best_set else 'None'}")
print(f"   Hybrid Cost: ${cost_best:,.2f}")
print(f"   Savings vs Decentralized: ${cost_q1 - cost_best:,.2f}")



 Q1 - Fully Decentralized Cost: $1,478,334.22

 Q2 - Full Centralization Cost (ρ = 0): $1,273,175.29
   Savings vs Decentralized: $205,158.93

 Q2 - Full Centralization Cost (ρ = 0.5): $1,434,195.98
   Savings vs Decentralized: $44,138.24

 Q2 - Full Centralization Cost (ρ = 1.0): $1,547,395.87
   Savings vs Decentralized: $-69,061.65

 Q3 - Hybrid Strategy (ρ = 0):
   Best categories to centralize: {'Medium', 'High', 'Low'}
   Hybrid Cost: $1,273,175.29
   Savings vs Decentralized: $205,158.93
